Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

In [ ]:
import numpy as np
import pandas as pd
import os
import sqlite3
import string

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Loading the data from SQLITE database

In [ ]:
os.chdir("C:/Users/admin/Desktop/amazon/")

In [ ]:
conn = sqlite3.connect("database.sqlite")
filtered_data = pd.read_sql_query("select * from Reviews where Score != 3 limit 15000",conn)

In [ ]:
filtered_data

In [ ]:
filtered_data.shape

In [ ]:
def partion(x):
    if x < 3:
        return 0
    return 1

In [ ]:
actualscore = filtered_data.loc[:,"Score"]
posnegscore = actualscore.map(partion)
filtered_data["Score"] = posnegscore

# Exploratary Data Analysis

* Remove Dublicate entries where user id,Reviews and Scores are the same
* Remove the Entry where HelpfulnessNumerator is greater than HelpfulnessDenominator

In [ ]:
sorted_data = filtered_data.sort_values("UserId",axis=0,ascending=True,inplace=False,kind="quicksort",na_position='last')

In [ ]:
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep="first",inplace=False)
final.shape

In [ ]:
# Percentage of remaining data
(final["Id"].size * 1) / (filtered_data["Id"].size * 1) * 100

In [ ]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final.shape

# Text-Preprocessing

1. We will remove HTML tags,punctuations and Special character
2. Check if the word is made up of english letters and is not alpha-numeric
3. Check if length of word is greater than 2
4. convert to lower letter
5. remove stop-words
6. Snowball stemming the word

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
stop = set(stopwords.words("english"))

In [ ]:
set(stop)

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
sno = nltk.SnowballStemmer("english")

In [ ]:
import re
def cleanhtml(sentence):
    clean = re.compile("<.*?>")
    cleantext = re.sub(clean," ",sentence)
    return cleantext
def cleanpunc(sentence):
    cleantext = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleantext = re.sub(r'[.|,|)|(|\|/]',r'',cleantext)
    return cleantext

In [ ]:
i = 0
final_string = []
postive_string = []
negative_string = []
s = ""

for sent in final["Text"].values:
    filter_string = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for clean in cleanpunc(w).split():
            if (clean.isalpha()) & (len(clean) > 2):
                if(clean.lower() not in stop):
                    s = (sno.stem(clean.lower())).encode("utf8")
                    filter_string.append(s)
                    if (final["Score"].values)[i] == 1:
                        postive_string.append(s)
                    else:
                        negative_string.append(s)
                    
                else:
                    continue
                
                
            else:
                continue
    str1 = b"".join(filter_string)
    final_string.append(str1)
    i+=1

In [ ]:
final["CleanText"] = final_string

In [ ]:
final.head()

In [ ]:
conn = sqlite3.connect("final.sqlite")
conn.text_factory = str
final.to_sql("Reviews",conn,if_exists="replace")

# Converting cleaned text to vectors

1. Bag of words
2. Word2Vec

In [ ]:
#Bag of words (BOW)
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vect = CountVectorizer()
final_count_vect = count_vect.fit_transform(final["CleanText"].values)

In [ ]:
final_count_vect.shape

In [ ]:
##Word2Vec

In [ ]:
import gensim

In [ ]:
list_sent = []
for sent in final["CleanText"].values:
    filter_sent =[]
    sent = cleanhtml(sent)
    for w in sent.split():
        for clean_words in cleanpunc(w).split():
            if (clean_words.isalpha()):
                filter_sent.append(clean_words.lower())
            else:
                continue
    list_sent.append(filter_sent)

In [ ]:
w2v_model = gensim.models.Word2Vec(list_sent,min_count=1,max_vocab_size=50,workers=4)

In [ ]:
words = list(w2v_model.wv.vocab)